In [1]:
import contexttimer
from runlmc.models.lmc import LMC
from runlmc.kern.rbf import RBF
from runlmc.models.optimization import AdaDelta
import GPy
from runlmc.models.gpy_lmc import GPyLMC

import numpy as np

%matplotlib inline 
import matplotlib.pyplot as plt

np.random.seed(1234)

In [2]:
# Adapts the 2-output toy problem from 
# Collaborative Multi-output Gaussian Processes
# Nguyen and Bonilla et al.

# Their example uses a grid of inputs. To make it harder (for runlmc)
# we instead look at uniformly distributed inputs.

sz = 2000
xss = [np.random.uniform(-10,10,size=sz) for _ in range(2)]
yss = [np.sin(xss[0]) + 1e-7 + np.random.randn(sz) * 1e-2,
       -np.sin(xss[0]) + 1e-7 + np.random.randn(sz) * 1e-2]
ks = [RBF(name='rbf0')]

In [3]:
lmc = LMC(xss, yss, kernels=ks, normalize=False)
opt = AdaDelta(max_it=100, verbosity=10)
with contexttimer.Timer() as t:
    lmc.optimize(optimizer=opt)
t.elapsed, lmc.log_likelihood()

starting AdaDelta {'max_it': 100, 'step_rate': 1, 'offset': 0.0001, 'momentum': 0.5, 'verbosity': 10, 'min_grad_ratio': 0.5, 'roll': 2, 'decay': 0.9}
iteration       10 grad norm 8.0037e+02
iteration       20 grad norm 8.3000e+02
finished adadelta optimization
            22 iterations
    3.3281e+02 final grad norm
    3.8924e+02 final MA(2) grad norm
    9.7431e+02 max MA(2) grad norm
    norm used inf


(20.42438423691783, 4155.6073917252616)

In [4]:
dtcvar = GPyLMC(xss, yss, kernels=ks, ranks=[1], sparse=10)
with contexttimer.Timer() as t:
    dtcvar.optimize()
t.elapsed, dtcvar.log_likelihood()

(24.23004325304646, 4150.727811393328)

In [11]:
# Cholesky
chol = GPyLMC(xss, yss, kernels=ks, ranks=[1])
with contexttimer.Timer() as t:
    chol.optimize()
t.elapsed, chol.log_likelihood()

(172.1126094950596, 4263.5540570123285)